In [3]:
import numpy as np
from collections import defaultdict

def optimized_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table):
    all_distances = defaultdict(lambda: np.inf)
    
    # Pre-compute all relevant keys and concept_ids
    all_keys = list(features.keys())
    all_concept_ids = set(concept_id for concept_ids in key_to_concept_ids.values() for concept_id in concept_ids)
    
    # Corrected vectorized ADC distance computation
    def vectorized_adc_distance(distance_table, codes):
        return np.sum(distance_table[np.arange(len(codes)), codes])
    
    for img_idx in matching_images:
        # Filter relevant concepts for this image
        relevant_concepts = [concept_idx for concept_idx in all_concept_ids if img_concept_bitmap[img_idx, concept_idx]]
        
        for key in all_keys:
            relevant_key_concepts = set(key_to_concept_ids[key]) & set(relevant_concepts)
            
            if not relevant_key_concepts:
                continue
            
            # Compute distances for all relevant concepts
            distances = []
            for concept_idx in relevant_key_concepts:
                pq_vector = packd[img_idx, concept_idx]
                distance_table = concept_d_table[key, concept_idx]
                distance = vectorized_adc_distance(distance_table, pq_vector)
                distances.append(distance)
            
            # Find the minimum distance across all concepts
            if distances:
                min_distance = min(distances)
                all_distances[img_idx, key] = min(all_distances[img_idx, key], min_distance)
    
    return dict(all_distances)

In [8]:
import numpy as np
from collections import defaultdict

def original_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table):
    all_distances = {}
    for img_idx in matching_images:
        for key in features:
            for concept_idx in key_to_concept_ids[key]:
                if not img_concept_bitmap[img_idx, concept_idx]:
                    continue

                # Calculate distances using ADC
                pq_vectors = np.array(packd[img_idx, concept_idx])
                distance_table = concept_d_table[key, concept_idx]
                adc_distances = np.sum(distance_table[np.arange(len(pq_vectors)), pq_vectors])

                if (img_idx, key) not in all_distances:
                    all_distances[(img_idx, key)] = adc_distances
                else:
                    if adc_distances < all_distances[(img_idx, key)]:
                        all_distances[(img_idx, key)] = adc_distances

    return all_distances

In [40]:
import numpy as np
from multiprocessing import Pool, shared_memory
from functools import partial

def create_shared_array(arr):
    shm = shared_memory.SharedMemory(create=True, size=arr.nbytes)
    shared_arr = np.ndarray(arr.shape, dtype=arr.dtype, buffer=shm.buf)
    shared_arr[:] = arr[:]
    return shm, shared_arr

def process_chunk(chunk_slice, shm_names, shapes, dtypes, key_to_concept_ids, all_keys):
    # Attach to shared memory
    img_concept_bitmap = np.ndarray(shapes['img_concept_bitmap'], dtypes['img_concept_bitmap'], 
                                    buffer=shared_memory.SharedMemory(name=shm_names['img_concept_bitmap']).buf)
    packd_array = np.ndarray(shapes['packd_array'], dtypes['packd_array'], 
                             buffer=shared_memory.SharedMemory(name=shm_names['packd_array']).buf)
    d_table_array = np.ndarray(shapes['d_table_array'], dtypes['d_table_array'], 
                               buffer=shared_memory.SharedMemory(name=shm_names['d_table_array']).buf)

    chunk_results = {}
    for img_idx in range(chunk_slice.start, chunk_slice.stop):
        valid_concepts = img_concept_bitmap[img_idx]
        for key_idx, key in enumerate(all_keys):
            relevant_concepts = [c for c in key_to_concept_ids[key] if valid_concepts[c]]
            if not relevant_concepts:
                continue
            
            codes = packd_array[img_idx, relevant_concepts]
            distances = np.sum(
                d_table_array[key_idx, relevant_concepts][
                    np.arange(len(relevant_concepts))[:, np.newaxis],
                    np.arange(d_table_array.shape[2]),
                    codes
                ],
                axis=1
            )
            chunk_results[img_idx, key] = np.min(distances)

    return chunk_results

def optimized_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table, n_processes=None, chunk_size=100):
    all_keys = list(features.keys())
    all_concept_ids = list(set(concept_id for concept_ids in key_to_concept_ids.values() for concept_id in concept_ids))
    
    # Convert packd and concept_d_table to numpy arrays
    max_img_idx = max(matching_images) + 1
    n_concepts = len(all_concept_ids)
    n_sub_vectors = next(iter(packd.values())).shape[0]
    n_keys = len(all_keys)
    n_centroids = next(iter(concept_d_table.values())).shape[1]

    packd_array = np.full((max_img_idx, n_concepts, n_sub_vectors), -1, dtype=int)
    for (img, concept), code in packd.items():
        packd_array[img, concept] = code

    d_table_array = np.zeros((n_keys, n_concepts, n_sub_vectors, n_centroids))
    for (key, concept), table in concept_d_table.items():
        d_table_array[all_keys.index(key), concept] = table

    # Create shared memory
    shm_img_concept_bitmap, shared_img_concept_bitmap = create_shared_array(img_concept_bitmap)
    shm_packd_array, shared_packd_array = create_shared_array(packd_array)
    shm_d_table_array, shared_d_table_array = create_shared_array(d_table_array)

    shm_names = {
        'img_concept_bitmap': shm_img_concept_bitmap.name,
        'packd_array': shm_packd_array.name,
        'd_table_array': shm_d_table_array.name
    }
    shapes = {
        'img_concept_bitmap': img_concept_bitmap.shape,
        'packd_array': packd_array.shape,
        'd_table_array': d_table_array.shape
    }
    dtypes = {
        'img_concept_bitmap': img_concept_bitmap.dtype,
        'packd_array': packd_array.dtype,
        'd_table_array': d_table_array.dtype
    }

    # Prepare chunks
    chunks = [range(i, min(i + chunk_size, len(matching_images))) 
              for i in range(0, len(matching_images), chunk_size)]

    # Process chunks in parallel
    with Pool(processes=n_processes) as pool:
        chunk_results = pool.map(
            partial(process_chunk, shm_names=shm_names, shapes=shapes, dtypes=dtypes, 
                    key_to_concept_ids=key_to_concept_ids, all_keys=all_keys),
            chunks
        )

    # Combine results
    all_distances = {}
    for chunk_result in chunk_results:
        all_distances.update(chunk_result)

    # Clean up shared memory
    shm_img_concept_bitmap.close()
    shm_img_concept_bitmap.unlink()
    shm_packd_array.close()
    shm_packd_array.unlink()
    shm_d_table_array.close()
    shm_d_table_array.unlink()

    return all_distances

In [41]:
import numpy as np
from collections import defaultdict
import time

def test_distance_computations():
    # Mock data (unchanged)
    matching_images = [0, 1, 2]
    features = {'key1': np.array([1, 2, 3]), 'key2': np.array([4, 5, 6])}
    key_to_concept_ids = {'key1': [0, 1], 'key2': [1, 2]}
    img_concept_bitmap = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [1, 0, 1]
    ])
    packd = {
        (0, 0): np.array([0, 1]),
        (0, 1): np.array([1, 0]),
        (1, 1): np.array([1, 1]),
        (1, 2): np.array([0, 0]),
        (2, 0): np.array([1, 1]),
        (2, 2): np.array([0, 1])
    }
    concept_d_table = {
        ('key1', 0): np.array([[1, 2], [3, 4]]),
        ('key1', 1): np.array([[2, 3], [4, 5]]),
        ('key2', 1): np.array([[3, 4], [5, 6]]),
        ('key2', 2): np.array([[4, 5], [6, 7]])
    }

    try:
        # Run all implementations and measure time
        start = time.time()
        original_result = original_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)
        original_time = time.time() - start

        start = time.time()
        optimized_result = optimized_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)
        optimized_time = time.time() - start

        # Compare results
        for key in original_result.keys():
            assert np.isclose(original_result[key], optimized_result[key]), f"Mismatch between original and optimized for key {key}"

        print("All tests passed! All implementations produce the same results.")

        # Print the results and timing for verification
        print("\nResults:")
        for key in sorted(original_result.keys()):
            print(f"{key}: {original_result[key]}")

        print(f"\nExecution times:")
        print(f"Original: {original_time:.6f} seconds")
        print(f"Optimized: {optimized_time:.6f} seconds")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Debug information:")
        print(f"matching_images: {matching_images}")
        print(f"features: {features}")
        print(f"key_to_concept_ids: {key_to_concept_ids}")
        print(f"img_concept_bitmap shape: {img_concept_bitmap.shape}")
        print(f"packd keys: {packd.keys()}")
        print(f"concept_d_table keys: {concept_d_table.keys()}")

# Run the test
test_distance_computations()

An error occurred: operation forbidden on released memoryview object
Debug information:
matching_images: [0, 1, 2]
features: {'key1': array([1, 2, 3]), 'key2': array([4, 5, 6])}
key_to_concept_ids: {'key1': [0, 1], 'key2': [1, 2]}
img_concept_bitmap shape: (3, 3)
packd keys: dict_keys([(0, 0), (0, 1), (1, 1), (1, 2), (2, 0), (2, 2)])
concept_d_table keys: dict_keys([('key1', 0), ('key1', 1), ('key2', 1), ('key2', 2)])


In [43]:
import numpy as np
from collections import defaultdict
import time
import random

def generate_stressful_data(n_images=1000, n_features=50, n_concepts=100, n_sub_vectors=8, n_centroids=256):
    matching_images = list(range(n_images))
    features = {f'key{i}': np.random.rand(128) for i in range(n_features)}
    key_to_concept_ids = {f'key{i}': random.sample(range(n_concepts), 10) for i in range(n_features)}
    
    img_concept_bitmap = np.random.choice([0, 1], size=(n_images, n_concepts), p=[0.7, 0.3])
    
    packd = {(i, j): np.random.randint(0, n_centroids, size=n_sub_vectors) 
             for i in range(n_images) for j in range(n_concepts) if img_concept_bitmap[i, j]}
    
    concept_d_table = {(f'key{i}', j): np.random.rand(n_sub_vectors, n_centroids) 
                       for i in range(n_features) for j in key_to_concept_ids[f'key{i}']}
    
    return matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table

def test_distance_computations():
    print("Generating stressful data...")
    matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table = generate_stressful_data()
    print("Data generation complete.")

    try:
        print("Running original implementation...")
        start = time.time()
        original_result = original_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)
        original_time = time.time() - start
        print(f"Original implementation completed in {original_time:.2f} seconds.")

        print("Running optimized implementation...")
        start = time.time()
        optimized_result = optimized_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)
        optimized_time = time.time() - start
        print(f"Optimized implementation completed in {optimized_time:.2f} seconds.")

        # Compare results
        print("Comparing results...")
        max_diff = 0
        for key in original_result.keys():
            diff = abs(original_result[key] - optimized_result[key])
            max_diff = max(max_diff, diff)
            assert diff < 1e-5, f"Significant mismatch between original and optimized for key {key}: {original_result[key]} vs {optimized_result[key]}"

        print(f"All tests passed! Maximum difference between implementations: {max_diff}")

        print(f"\nExecution times:")
        print(f"Original: {original_time:.6f} seconds")
        print(f"Optimized: {optimized_time:.6f} seconds")
        print(f"Speedup: {original_time / optimized_time:.2f}x")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Debug information:")
        print(f"Number of matching_images: {len(matching_images)}")
        print(f"Number of features: {len(features)}")
        print(f"Number of concepts: {img_concept_bitmap.shape[1]}")
        print(f"img_concept_bitmap shape: {img_concept_bitmap.shape}")
        print(f"Number of packd entries: {len(packd)}")
        print(f"Number of concept_d_table entries: {len(concept_d_table)}")
        raise  # Re-raise the exception to see the full traceback

# Run the test
test_distance_computations()

Generating stressful data...
Data generation complete.
Running original implementation...
Original implementation completed in 0.62 seconds.
Running optimized implementation...
An error occurred: operation forbidden on released memoryview object
Debug information:
Number of matching_images: 1000
Number of features: 50
Number of concepts: 100
img_concept_bitmap shape: (1000, 100)
Number of packd entries: 30008
Number of concept_d_table entries: 500


ValueError: operation forbidden on released memoryview object

In [10]:
import numpy as np
from collections import defaultdict

def test_optimized_distance_computation():
    # Mock data (unchanged)
    matching_images = [0, 1, 2]
    features = {'key1': np.array([1, 2, 3]), 'key2': np.array([4, 5, 6])}
    key_to_concept_ids = {'key1': [0, 1], 'key2': [1, 2]}
    img_concept_bitmap = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [1, 0, 1]
    ])
    packd = {
        (0, 0): np.array([0, 1]),
        (0, 1): np.array([1, 0]),
        (1, 1): np.array([1, 1]),
        (1, 2): np.array([0, 0]),
        (2, 0): np.array([1, 1]),
        (2, 2): np.array([0, 1])
    }
    concept_d_table = {
        ('key1', 0): np.array([[1, 2], [3, 4]]),
        ('key1', 1): np.array([[2, 3], [4, 5]]),
        ('key2', 1): np.array([[3, 4], [5, 6]]),
        ('key2', 2): np.array([[4, 5], [6, 7]])
    }

    # Run both implementations
    original_result = original_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)
    optimized_result = optimized_distance_computation(matching_images, features, key_to_concept_ids, img_concept_bitmap, packd, concept_d_table)

    # Compare results
    for key in original_result.keys():
        assert np.isclose(original_result[key], optimized_result[key]), f"Mismatch for key {key}: original {original_result[key]}, optimized {optimized_result[key]}"

    print("All tests passed! Optimized implementation matches the original.")

    # Print the results for verification
    print("\nResults:")
    for key in sorted(original_result.keys()):
        print(f"{key}: {original_result[key]}")

# Run the test
test_optimized_distance_computation()

All tests passed! Optimized implementation matches the original.

Results:
(0, 'key1'): 5
(0, 'key2'): 9
(1, 'key1'): 8
(1, 'key2'): 10
(2, 'key1'): 6
(2, 'key2'): 11
